In [1]:
import os

annotation_dir = os.path.join('fabric_stain_dataset', 'annotations', 'stain')
total_stains = 0
max_stains = 0

for annotation_file in os.listdir(annotation_dir):
    annotation_file = os.path.join(annotation_dir, annotation_file)
    with open(annotation_file, 'r') as file:
        num_stains = len(file.readlines())
        total_stains += num_stains
        if num_stains > max_stains:
            max_stains = num_stains

print("Total number of stains:", total_stains)
print("Maximum number of stains in a single image:", max_stains)

Total number of stains: 870
Maximum number of stains in a single image: 14


In [ ]:
import numpy as np
import shutil

#Because there are multiple stains per iamge, this gets a bit tricky
source_dir = os.path.join('fabric_stain_dataset', 'images', 'stain')
train_dir = os.path.join('fabric_stain_dataset', 'split_images', 'train')
val_dir = os.path.join('fabric_stain_dataset', 'split_images', 'val')
train_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'train')
val_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'val')
annotation_dir = os.path.join('fabric_stain_dataset', 'annotations', 'stain')

# Create the balanced dataset folders if it doesn't exist
def handle_dir_creation(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    else:
        # Remove all the files in the folder
        for file_name in os.listdir(dir):
            file_path = os.path.join(dir, file_name)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")

# Create the directories if they don't exist
handle_dir_creation(train_dir)
handle_dir_creation(val_dir)
handle_dir_creation(train_annotation_dir)
handle_dir_creation(val_annotation_dir)

# Ensure the target directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(train_annotation_dir, exist_ok=True)
os.makedirs(val_annotation_dir, exist_ok=True)


#We'll be using an 80-20 split for the training and validating datasets. this is pretty standard.
train_test_split = 0.8 

image_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# Shuffle the list to ensure randomness
np.random.shuffle(image_files)

#Find the total number of stains in the dataset
total_stains = 0
for annotation_file in os.listdir(annotation_dir):
    annotation_file = os.path.join(annotation_dir, annotation_file)
    with open(annotation_file, 'r') as file:
        num_stains = len(file.readlines())
        total_stains += num_stains

num_train_stains = int(total_stains * 0.8)
current_train_stains = 0


for img in image_files:
    # Get the annotation file for the image
    annotation_file = os.path.join(annotation_dir, img.replace('.jpg', '.txt'))
    with open(annotation_file, 'r') as file:
        num_stains = len(file.readlines())
        if current_train_stains + num_stains <= num_train_stains:
            current_train_stains += num_stains
            shutil.copy(os.path.join(source_dir, img), os.path.join(train_dir, img))
            shutil.copy(annotation_file, os.path.join(train_annotation_dir, os.path.basename(annotation_file)))
        else:
            shutil.copy(os.path.join(source_dir, img), os.path.join(val_dir, img))
            shutil.copy(annotation_file, os.path.join(val_annotation_dir, os.path.basename(annotation_file)))

num_train_imgs = len(os.listdir(train_dir))
num_val_imgs = len(os.listdir(val_dir))

print(f"Moved {num_train_imgs} images with {current_train_stains} stains to {train_dir}")
print(f"Moved {num_val_imgs} images with {total_stains - current_train_stains} to {val_dir}")
print(f"Train val split is {current_train_stains / total_stains * 100:.2f}% - {(total_stains - current_train_stains) / total_stains * 100:.2f}%")

Moved 319 images with 696 stains to fabric_stain_dataset/split_images/train
Moved 79 images with 174 to fabric_stain_dataset/split_images/val
Train val split is 80.00% - 20.00%


In [ ]:
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

# Directories
train_dir = os.path.join('fabric_stain_dataset', 'split_images', 'train')
val_dir = os.path.join('fabric_stain_dataset', 'split_images', 'val')
train_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'train')
val_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'val')

# Function to load annotations
def load_annotations(annotation_file):
    annotations = []
    with open(annotation_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            label = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            annotations.append([label, x_center, y_center, width, height])
    return np.array(annotations)

# Function to preprocess data
def preprocess_data(image_dir, annotation_dir, image_size=(32, 32)):
    data = []
    annotations = []
    max_annotations = 0
    for img_name in os.listdir(image_dir):
        if img_name.endswith('.jpg'):
            image_path = os.path.join(image_dir, img_name)
            annotation_path = os.path.join(annotation_dir, img_name.replace('.jpg', '.txt'))
            
            if os.path.isfile(annotation_path):
                image = Image.open(image_path)
                image = image.resize(image_size)  # Resize image
                image = image.convert('L')  # Convert to grayscale
                image = np.array(image) / 255.0  # Normalize the image
                image = np.expand_dims(image, axis=-1)  # Add channel dimension
                annotation = load_annotations(annotation_path)
                
                data.append(image)
                annotations.append(annotation)
                
                # Update the maximum number of annotations
                if annotation.shape[0] > max_annotations:
                    max_annotations = annotation.shape[0]
    
    return np.array(data), annotations, max_annotations

# Preprocess training and validation data
train_data, train_annotations, train_max_annotations = preprocess_data(train_dir, train_annotation_dir)
val_data, val_annotations, val_max_annotations = preprocess_data(val_dir, val_annotation_dir)

# Determine the global maximum number of annotations
global_max_annotations = max(train_max_annotations, val_max_annotations)

# Pad annotations to the global maximum length
def pad_annotations(annotations, max_annotations):
    padded_annotations = []
    for annotation in annotations:
        if annotation.shape[0] < max_annotations:
            padding = np.zeros((max_annotations - annotation.shape[0], annotation.shape[1]))
            padded_annotation = np.vstack((annotation, padding))
        else:
            padded_annotation = annotation
        padded_annotations.append(padded_annotation)
    return np.array(padded_annotations)

train_annotations_padded = pad_annotations(train_annotations, global_max_annotations)
val_annotations_padded = pad_annotations(val_annotations, global_max_annotations)

# Flatten the annotations to match the model output
train_annotations_flat = train_annotations_padded.reshape((train_annotations_padded.shape[0], -1))
val_annotations_flat = val_annotations_padded.reshape((val_annotations_padded.shape[0], -1))

print("Shape of train_annotations_flat:", train_annotations_flat.shape)
print("Shape of val_annotations_flat:", val_annotations_flat.shape)

print("Data preprocessing completed")

# Define a more suitable model for handling bounding boxes
print("Building the model")
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(train_annotations_flat.shape[1], activation='sigmoid'))
print("Model built")

model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use appropriate loss function
              metrics=['accuracy'])
print("Model compiled")

# Print the model summary
print("Model Summary:")
model.summary()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the model using the augmented data generator
print("Training the model with data augmentation")
history = model.fit(datagen.flow(train_data, train_annotations_flat, batch_size=32),
                    steps_per_epoch=len(train_data) // 32,
                    epochs=9,
                    validation_data=(val_data, val_annotations_flat))

# Print the training metrics
print("Training Metrics:")
for epoch in range(len(history.history['loss'])):
    print(f"Epoch {epoch + 1}:")
    print(f"  Loss: {history.history['loss'][epoch]}")
    print(f"  Accuracy: {history.history['accuracy'][epoch]}")
    print(f"  Validation Loss: {history.history['val_loss'][epoch]}")
    print(f"  Validation Accuracy: {history.history['val_accuracy'][epoch]}")

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(val_data, val_annotations_flat, verbose=2)

# Print the validation metrics
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predict on validation data
val_predictions = model.predict(val_data)

# Calculate Intersection over Union (IoU)
def calculate_iou(true_boxes, pred_boxes):
    ious = []
    for true_box, pred_box in zip(true_boxes, pred_boxes):
        _, x1_true, y1_true, w_true, h_true = true_box  # Unpack the true box
        _, x1_pred, y1_pred, w_pred, h_pred = pred_box  # Unpack the predicted box
        
        x1_true, y1_true = x1_true - w_true / 2, y1_true - h_true / 2
        x2_true, y2_true = x1_true + w_true, y1_true + h_true
        x1_pred, y1_pred = x1_pred - w_pred / 2, y1_pred - h_pred / 2
        x2_pred, y2_pred = x1_pred + w_pred, y1_pred + h_pred
        
        xi1, yi1 = max(x1_true, x1_pred), max(y1_true, y1_pred)
        xi2, yi2 = min(x2_true, x2_pred), min(y2_true, y2_pred)
        
        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
        true_area = (x2_true - x1_true) * (y2_true - y1_true)
        pred_area = (x2_pred - x1_pred) * (y2_pred - y1_pred)
        
        union_area = true_area + pred_area - inter_area
        iou = inter_area / union_area if union_area != 0 else 0
        ious.append(iou)
    return np.mean(ious)

# Calculate IoU for validation data
ious = []
for i in range(len(val_annotations_flat)):
    true_boxes = val_annotations_flat[i].reshape(-1, 5)
    pred_boxes = val_predictions[i].reshape(-1, 5)
    iou = calculate_iou(true_boxes, pred_boxes)
    ious.append(iou)

mean_iou = np.mean(ious)
print(f"Mean IoU: {mean_iou}")

# Save the model
model.save('simple_cnn_model.h5')

Results taken from Google Collab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape of train_annotations_flat: (319, 70)
Shape of val_annotations_flat: (79, 70)
Data preprocessing completed
Building the model
Model built
Model compiled
Model Summary:
Model: "sequential_13"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_39 (Conv2D)                   │ (None, 30, 30, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_39               │ (None, 30, 30, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_39 (MaxPooling2D)      │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_40 (Conv2D)                   │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_40               │ (None, 13, 13, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_40 (MaxPooling2D)      │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_41 (Conv2D)                   │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_41               │ (None, 4, 4, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_41 (MaxPooling2D)      │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_13 (Flatten)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 70)                  │          17,990 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 505,542 (1.93 MB)
 Trainable params: 505,094 (1.93 MB)
 Non-trainable params: 448 (1.75 KB)
Training the model with data augmentation
Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.0672 - loss: 0.2106 - val_accuracy: 0.2278 - val_loss: 0.1812
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0938 - loss: 0.0653 - val_accuracy: 0.2278 - val_loss: 0.1724
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.1820 - loss: 0.0450 - val_accuracy: 0.1772 - val_loss: 0.0970
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0938 - loss: 0.0233 - val_accuracy: 0.2278 - val_loss: 0.0907
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.1809 - loss: 0.0250 - val_accuracy: 0.2658 - val_loss: 0.0528
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2500 - loss: 0.0196 - val_accuracy: 0.2658 - val_loss: 0.0499
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.2083 - loss: 0.0234 - val_accuracy: 0.2658 - val_loss: 0.0315
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4375 - loss: 0.0179 - val_accuracy: 0.2658 - val_loss: 0.0301
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.2604 - loss: 0.0225 - val_accuracy: 0.3165 - val_loss: 0.0223
Epoch 10/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1250 - loss: 0.0221 - val_accuracy: 0.2785 - val_loss: 0.0218
Training Metrics:
Epoch 1:
  Loss: 0.15669475495815277
  Accuracy: 0.08013937622308731
  Validation Loss: 0.18115410208702087
  Validation Accuracy: 0.2278480976819992
Epoch 2:
  Loss: 0.06529563665390015
  Accuracy: 0.09375
  Validation Loss: 0.17244872450828552
  Validation Accuracy: 0.2278480976819992
Epoch 3:
  Loss: 0.037273991852998734
  Accuracy: 0.16724738478660583
  Validation Loss: 0.09699070453643799
  Validation Accuracy: 0.17721518874168396
Epoch 4:
  Loss: 0.02330152317881584
  Accuracy: 0.09375
  Validation Loss: 0.09067019820213318
  Validation Accuracy: 0.2278480976819992
Epoch 5:
  Loss: 0.02430378459393978
  Accuracy: 0.20209059119224548
  Validation Loss: 0.05283856764435768
  Validation Accuracy: 0.26582279801368713
Epoch 6:
  Loss: 0.019597254693508148
  Accuracy: 0.25
  Validation Loss: 0.049933385103940964
  Validation Accuracy: 0.26582279801368713
Epoch 7:
  Loss: 0.023313453420996666
  Accuracy: 0.24041812121868134
  Validation Loss: 0.03150640428066254
  Validation Accuracy: 0.26582279801368713
Epoch 8:
  Loss: 0.017877992242574692
  Accuracy: 0.4375
  Validation Loss: 0.030076123774051666
  Validation Accuracy: 0.26582279801368713
Epoch 9:
  Loss: 0.022600959986448288
  Accuracy: 0.24738675355911255
  Validation Loss: 0.022342901676893234
  Validation Accuracy: 0.3164556920528412
Epoch 10:
  Loss: 0.02209358662366867
  Accuracy: 0.125
  Validation Loss: 0.021788330748677254
  Validation Accuracy: 0.27848100662231445
3/3 - 0s - 18ms/step - accuracy: 0.2785 - loss: 0.0218
Validation Loss: 0.021788330748677254
Validation Accuracy: 0.27848100662231445
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
Mean IoU: 0.0037942546209400346